# NeuralProphet Model for Cabbage Price Forecasting

This notebook implements NeuralProphet, a neural network-based time series forecasting model inspired by Facebook Prophet but implemented with PyTorch.

**Note:** NeuralProphet has dependency conflicts with some packages. This separate notebook allows you to run it independently.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

# Load the dataset
df_clean = pd.read_csv('data/df_clean.csv', index_col=0, parse_dates=True)
forecast_horizon = 7

print(f"Dataset loaded: {df_clean.shape}")
df_clean.head()

## Install NeuralProphet

First, we need to install NeuralProphet and its dependencies.

In [ ]:
# Install NeuralProphet
try:
    from neuralprophet import NeuralProphet
    print("NeuralProphet already installed!")
except ImportError:
    print("Installing NeuralProphet...")
    import sys
    !{sys.executable} -m pip install neuralprophet
    from neuralprophet import NeuralProphet
    print("NeuralProphet installed successfully!")

# Suppress NeuralProphet logging
import logging
logging.getLogger('neuralprophet').setLevel(logging.ERROR)

## Prepare Data for NeuralProphet

NeuralProphet requires data in Prophet format with 'ds' (date) and 'y' (target) columns.

In [ ]:
# Prepare data for NeuralProphet
prophet_df = pd.DataFrame({
    'ds': df_clean.index,
    'y': df_clean['price_per_kg'].values
})

# Remove any NaN values
prophet_df = prophet_df.dropna()

# Use 80/20 train/test split
prophet_train_size = int(len(prophet_df) * 0.8)
prophet_train = prophet_df.iloc[:prophet_train_size].copy()
prophet_test = prophet_df.iloc[prophet_train_size:].copy()

print(f"Training set: {len(prophet_train)} samples")
print(f"Test set: {len(prophet_test)} samples")
print(f"\nTrain date range: {prophet_train['ds'].iloc[0]} to {prophet_train['ds'].iloc[-1]}")
print(f"Test date range: {prophet_test['ds'].iloc[0]} to {prophet_test['ds'].iloc[-1]}")

## Train NeuralProphet Model

Train a Neural Prophet model with optimized parameters.

In [ ]:
# Initialize NeuralProphet model
model = NeuralProphet(
    epochs=100,
    batch_size=32,
    learning_rate=0.01,
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False,
    n_lags=7,  # Use past 7 days
    n_forecasts=7  # Forecast 7 days ahead
)

print("Training NeuralProphet model...")
print("This may take several minutes...\n")

# Train the model
metrics = model.fit(prophet_train, freq='D')

print("\n✓ NeuralProphet model trained successfully!")

## Make Predictions

In [ ]:
# Create future dataframe and make predictions
future = model.make_future_dataframe(prophet_train, periods=len(prophet_test))
forecast = model.predict(future)

print(f"Generated {len(forecast)} forecasts")
print(f"\nForecast columns: {forecast.columns.tolist()}")
forecast.tail(10)

## Evaluate Model Performance

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    """Calculate MAPE metric"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Extract test predictions
test_forecast = forecast.tail(len(prophet_test))

# Get actual values
y_true = prophet_test['y'].values
y_pred = test_forecast['yhat1'].values  # Day 1 forecast

# Calculate metrics
mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mape = mean_absolute_percentage_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

print("="*60)
print("NEURALPROPHET - MODEL EVALUATION")
print("="*60)
print(f"MAE:  {mae:.2f} KRW")
print(f"RMSE: {rmse:.2f} KRW")
print(f"MAPE: {mape:.2f}%")
print(f"R²:   {r2:.4f}")
print("="*60)

## Visualize Results

In [ ]:
# Plot predictions vs actuals
plt.figure(figsize=(16, 6))
plt.plot(prophet_test['ds'], y_true, label='Actual', linewidth=2, alpha=0.7)
plt.plot(test_forecast['ds'], y_pred, label='Predicted', linewidth=2, alpha=0.7, linestyle='--')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price per kg (KRW)', fontsize=12)
plt.title(f'NeuralProphet: Predictions vs Actual\nMAE: {mae:.2f}, RMSE: {rmse:.2f}, MAPE: {mape:.2f}%, R²: {r2:.4f}', 
          fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Save Model and Results

In [ ]:
import os
import pickle

# Create directories
os.makedirs('results', exist_ok=True)
os.makedirs('saved_models', exist_ok=True)

# Save results to CSV
results_df = pd.DataFrame({
    'Model': ['NeuralProphet'],
    'MAE': [mae],
    'RMSE': [rmse],
    'MAPE': [mape],
    'R2': [r2]
})
results_df.to_csv('results/neuralprophet_metrics.csv', index=False)
print("✓ Saved: results/neuralprophet_metrics.csv")

# Save predictions
predictions_df = pd.DataFrame({
    'Date': prophet_test['ds'],
    'Actual': y_true,
    'Predicted': y_pred
})
predictions_df.to_csv('results/neuralprophet_predictions.csv', index=False)
print("✓ Saved: results/neuralprophet_predictions.csv")

# Save model
with open('saved_models/neuralprophet_model.pkl', 'wb') as f:
    pickle.dump(model, f)
print("✓ Saved: saved_models/neuralprophet_model.pkl")

print("\n" + "="*60)
print("ALL NEURALPROPHET RESULTS AND MODEL SAVED!")
print("="*60)